<a href="https://colab.research.google.com/github/MarioCass10/Data-Science-Final-Project/blob/main/MasterProyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importamos librerias
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import graphviz
from sklearn import tree 

#Cargamos datos
master = pd.read_excel("/content/MasterProyecto.xlsx")

#Se eliminan filas que tengan valor NaN.
master = master.dropna()
master = master.reset_index(drop=True)

#Cabio el nombre a mis variables

master.rename(columns={"varprim_variacion_ipritari_ultimos_3_meses":"meses3","varprim_variacion_ipritari_ultimos_6_meses":"meses6","varprim_variacion_ipritari_ultimos_12_meses":"meses12"},inplace=True)


#convertimos de factor a numerico

'''conversortype1= preprocessing.LabelEncoder()
conversortype1.fit(master.tipo1.astype(str))
master["Tipo1numero"]= conversortype1.transform(master.tipo1.astype(str))

conversortype1= preprocessing.LabelEncoder()
conversortype1.fit(master.tipo2.astype(str))
master["Tipo2numero"]= conversortype1.transform(master.tipo2.astype(str))'''


# dividimos en variable a predecir (y) y variables predictoras (x)
y = master.loc[:,["target"]]
x = master.loc[:,~ master.columns.isin (['target',"meses3","meses6","meses12","#"])]

# dividimos en train (80%) y test (20%)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)


In [ ]:
from sklearn import tree 
#Entrenamos el modelo y vemos las variables mas importantes y hacemos un grafico para ello
model= tree.DecisionTreeClassifier(max_depth=3)
model.fit(x_train,y_train)
model.feature_importances_

def create_df():
    data={"nombres":x.columns,"Important":model.feature_importances_}
    df= pd.DataFrame(data)
    return df

variables_importantes= create_df()

from plotnine import *
ggplot(aes(x="nombres",y="Important",fill="Important"),variables_importantes)\
    +geom_bar(stat="identity",position="stack")+coord_flip()


#Vemos cómo es de bueno el modelo viendo cómo predice en el test
model.score(x_test,y_test)
predictions = model.predict(x_test)
confusion_matrix(y_test,predictions)

#Vemos cómo es el Arbol en el train con un gráfico de árbol
dot_data = export_graphviz(model, filled=True, feature_names=x_train.columns,
                           class_names=["No meses","Meses"])
graph = graphviz.Source(dot_data)
graph.render("master tree")

'master tree.pdf'